In [4]:
import tensorflow as tf
# import necessary packages
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.utils import get_file
import numpy as np
import argparse
import cv2
import os
import cvlib as cv
import matplotlib.pyplot as plt
import glob

# download pre-trained model file (one-time download)
dwnld_link = "https://github.com/arunponnusamy/cvlib/releases/download/v0.2.0/gender_detection.model"
model_path = get_file("gender_detection.model", dwnld_link,
                     cache_subdir="pre-trained", cache_dir=os.getcwd())



In [20]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm import tqdm, tqdm_notebook
import os
import time
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

In [67]:
model = load_model(model_path)

In [12]:
classes = ['man','woman']

In [66]:
model_res = ResNet50(weights='imagenet', include_top=False,
                 input_shape=(224, 224, 3))
def extract_features(img_path, model):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path, target_size=(
        input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

/Users/shivamralli/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [68]:
def man_or_woman(img):
    image = cv2.imread(img)
    face, confidence = cv.detect_face(image)
    for idx, f in enumerate(face):

         # get corner points of face rectangle       
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # draw rectangle over face
        #cv2.rectangle(image, (startX,startY), (endX,endY), (0,255,0), 2)

        # crop the detected face region
        face_crop = np.copy(image[startY:endY,startX:endX])

        # preprocessing for gender detection model
        face_crop = cv2.resize(face_crop, (96,96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # apply gender detection on face
        conf = model.predict(face_crop)[0]
        idx = np.argmax(conf)
        print(idx)
        if(classes[idx]=='man'):
            return 'man'
        elif(classes[idx]=='woman'):
            return 'woman'

In [148]:
img_to_search = 'data/missing_images/realigned/2.png'

In [149]:
val = man_or_woman(img_to_search)

1


In [150]:
val

'woman'

In [151]:
if val=='woman':
    filenames = pickle.load(open('resnet_filenames_woman.pickle', 'rb'))
    feature_list = pickle.load(open('resnet_features_woman.pickle', 'rb'))
elif val=='man':
    filenames = pickle.load(open('resnet_filenames_man.pickle', 'rb'))
    feature_list = pickle.load(open('resnet_features_man.pickle', 'rb'))

In [152]:
feature_of_img = extract_features(img_to_search, model_res)

## Getting n nearest neighbors

In [154]:
os.mkdir('search_space')

In [155]:
%%time

from sklearn.neighbors import NearestNeighbors
neighbors = NearestNeighbors(n_neighbors=100, algorithm='brute',
metric='euclidean').fit(feature_list)
distances, indices = neighbors.kneighbors([feature_of_img])

CPU times: user 643 ms, sys: 228 ms, total: 871 ms
Wall time: 1.09 s


In [156]:
indices

array([[ 82, 210, 258,  23,  45,  55, 251, 124,  49, 231,  34, 253,  59,
        125, 256, 103, 203, 238, 207, 239,  99, 235,  89, 119, 197,  29,
        247, 109, 227, 194, 111,  38, 205, 221,  63, 146, 248, 100, 116,
        224, 138, 250,  47,  44, 118, 244,  90, 192, 223,  75, 260,  32,
        106,  95,  39, 255, 236,  27,  97, 145, 222, 150,  50, 242, 104,
         51, 228, 229, 157, 122,  24, 219,  80, 261,  69,  94, 245, 241,
        198, 220, 202, 152, 225,  84, 237, 193, 117, 212, 128, 158,  40,
         92, 214, 246, 217, 252, 160, 127, 121, 200]])

In [157]:
filenames[indices[0][1]]

'woman/ArrestPersonArrestPerson1936.png'

In [158]:
for i in range(len(indices[0])):
    img_name = filenames[indices[0][i]]
    img = cv2.imread(img_name)
    if val=='woman':
        sp = 'search_space/'+img_name[6:]
    elif val=='man':
        sp = 'search_space/'+img_name[4:]
    #print(sp)
    cv2.imwrite(sp,img)